# Analysis

This notebook contains routines for analyzing the output of keypoint-MoSeq.

```{note}
For the widgets below to work in a jupyter notebook, you must launch jupyter from a terminal in which the keypoint_moseq environment is active.
```

## Setup

We assume you have already have keypoint-MoSeq outputs that are organized as follows.
```
<project_dir>/               ** current working directory
└── <model_name>/            ** model directory
    ├── results.h5           ** model results
    └── grid_movies/         ** [Optional] grid movies folder
```
Use the code below to define your project directory and model name, and to check if the required jupyter extensions are enabled. If they are not, run the [installation commands](https://keypoint-moseq.readthedocs.io/en/latest/install.html#install-jupyter-extensions) and confirm that jupyter was launched from the `keypoint_moseq` environment, as mentioned in the note above. 

If the required jupyter extensions doesn't load corectly, you won't be able to use the interactive widgets but you could continue with other analysis components using the non-interactive cells.


In [2]:
import keypoint_moseq as kpms

# project_dir='path/to/project' # the full path to the project directory
# model_name='model_name' # name of model to analyze (e.g. something like `2023_05_23-15_19_03`)

project_dir='/Users/sherry/Desktop/kpms_dev/demo_project' # the full path to the project directory
model_name='2023_09_08-17_58_58' # name of model to analyze (e.g. something like `2023_05_23-15_19_03`)


## Assign Groups

**Interactive widget**

The following cell invokes an interactive spreadsheet widget that can be used to annotate each recording with a group label. These labels are important later for performing group-wise comparisons. The annotations are saved to a to  `{project_dir}/index.yaml`.

- Select one or more rows, enter a group name and click `Set Group Name`
- Click any column header to sort rows alphabetically
- Click the filter icon in a column header to filter rows by name. 
- At any point, use `Update Index File` to save current group assignments.

In [2]:
kpms.interactive_group_setting(project_dir, model_name)

Row
    [0] Tabulator(editors={'name': None, ...}, show_index=False, titles={'name': 'file name', ...}, value=              ..., widths={'name': 400})
    [1] Column
        [0] Button(button_type='primary', name='Update Index File')

**Non-interactive alternative**

The followign two cells provide a non-interactive alternative to the above widget. The first cell creates a csv with the same columns as the widget. The csv file is saved to `{project_dir}/index.csv`.

You could use Excel or LibreOffice Calc to edit the csv. The second cell reads the csv and saves the group assignments to the index file.

```{note}


In [ ]:
# create index dataframe and save it to csv
index_df = kpms.create_index_file(project_dir, model_name)
# display first 5 row of index dataframe
index_df.head()

In [ ]:
# update information from csv to index file
kpms.update_index_file(project_dir, model_name)

## Generate dataframes

Generate a pandas dataframe called `moseq_df` that contains syllable labels and kinematic information for each frame across all the recording sessions. 

In [3]:
moseq_df = kpms.compute_moseq_df(project_dir, model_name, smooth_heading=True) 
moseq_df

,name,centroid_x,centroid_y,heading,angular_velocity,velocity_px_s,syllable,frame_index,group,onset
0,21_11_8_one_mouse.top.irDLC_resnet50_moseq_exa...,250.185319,208.132543,-1.225495,0.000000,0.000000,5,0,sdf,True
1,21_11_8_one_mouse.top.irDLC_resnet50_moseq_exa...,249.387490,208.141737,-1.225495,-0.105045,23.936449,5,1,sdf,False
2,21_11_8_one_mouse.top.irDLC_resnet50_moseq_exa...,248.827569,207.422977,-1.232140,-0.204673,27.333404,5,2,sdf,False
3,21_11_8_one_mouse.top.irDLC_resnet50_moseq_exa...,248.601599,205.904871,-1.271636,-0.296745,46.044935,5,3,sdf,False
4,21_11_8_one_mouse.top.irDLC_resnet50_moseq_exa...,247.723307,203.228498,-1.271636,-0.383430,84.504054,5,4,sdf,False
...,...,...,...,...,...,...,...,...,...,...
643906,22_27_04_cage4_mouse2_0.top.irDLC_resnet50_mos...,154.266773,144.784688,0.723165,-0.170320,409.395558,23,53618,sdf,False
643907,22_27_04_cage4_mouse2_0.top.irDLC_resnet50_mos...,143.945329,134.288740,0.675987,-0.169697,441.619093,23,53619,sdf,False
643908,22_27_04_cage4_mouse2_0.top.irDLC_resnet50_mos...,133.754331,124.082025,0.675987,-0.147606,432.700975,23,53620,sdf,False
643909,22_27_04_cage4_mouse2_0.top.irDLC_resnet50_mos...,124.364825,113.735113,0.675987,-0.107560,419.164942,23,53621,sdf,False


In [4]:
moseq_df.group.unique()

array(['sdf', 'dfd'], dtype=object)

In [ ]:
import os
save_dir = os.path.join(project_dir, model_name) # directory to save the moseq_df dataframe
moseq_df.to_csv(os.path.join(save_dir, 'moseq_df.csv'), index=False)
print('Saved `moseq_df` dataframe to', save_dir)

Next generate a dataframe called `stats_df` that contains summary statistics for each syllable in each recording session, such as its usage frequency and its distribution of kinematic parameters.

In [9]:
stats_df = kpms.compute_stats_df(
    project_dir,
    model_name,
    moseq_df, 
    min_frequency=0.005,       # threshold frequency for including a syllable in the dataframe
    groupby=[], # column(s) to group the dataframe by
    fps=30)                    # frame rate of the video from which keypoints were inferred

stats_df

,syllable,heading_mean,heading_std,heading_min,heading_max,angular_velocity_mean,angular_velocity_std,angular_velocity_min,angular_velocity_max,velocity_px_s_mean,velocity_px_s_std,velocity_px_s_min,velocity_px_s_max,group,frequency,duration
0,0,0.001052,1.751396,-3.141575,3.141386,0.038213,7.033167,-188.447819,188.450313,26.158692,22.418688,0.088198,249.883525,sdf,0.114184,0.788430
1,0,0.001052,1.751396,-3.141575,3.141386,0.038213,7.033167,-188.447819,188.450313,26.158692,22.418688,0.088198,249.883525,sdf,0.093969,0.788430
2,0,0.001052,1.751396,-3.141575,3.141386,0.038213,7.033167,-188.447819,188.450313,26.158692,22.418688,0.088198,249.883525,sdf,0.121605,0.788430
3,0,0.001052,1.751396,-3.141575,3.141386,0.038213,7.033167,-188.447819,188.450313,26.158692,22.418688,0.088198,249.883525,dfd,0.124543,0.788430
4,0,0.001052,1.751396,-3.141575,3.141386,0.038213,7.033167,-188.447819,188.450313,26.158692,22.418688,0.088198,249.883525,sdf,0.111344,0.788430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,17,0.059771,1.782905,-3.116595,3.140159,-0.585299,22.581862,-188.098556,182.802028,45.888822,60.467227,2.306052,491.208755,sdf,0.003691,0.087841
176,17,0.059771,1.782905,-3.116595,3.140159,-0.585299,22.581862,-188.098556,182.802028,45.888822,60.467227,2.306052,491.208755,dfd,0.000000,0.087841
177,17,0.059771,1.782905,-3.116595,3.140159,-0.585299,22.581862,-188.098556,182.802028,45.888822,60.467227,2.306052,491.208755,sdf,0.004909,0.087841
178,17,0.059771,1.782905,-3.116595,3.140159,-0.585299,22.581862,-188.098556,182.802028,45.888822,60.467227,2.306052,491.208755,dfd,0.004670,0.087841


In [ ]:
import os
save_dir = os.path.join(project_dir, model_name)
stats_df.to_csv(os.path.join(save_dir, 'stats_df'), index=False)
print('Saved `stats_df` dataframe to', save_dir)

##  Label syllables

**Interactive widget**

The following cell invoke an interactive widget to assign a name and short description to each syllable for downstream interpretation. These annotations are saved to `{project_dir}/{model_name}/syll_info.yaml`.

In [ ]:
kpms.label_syllables(project_dir, model_name, moseq_df) 

**Non-interactive alternative**

The followign two cells provide a non-interactive alternative to the above widget. The first cell creates a csv for you to fill the syllable name and a short description. The csv file is saved to `{project_dir}/syll_info.csv`.

You could use Excel or LibreOffice Calc to edit the csv. The second cell reads the csv and saves the syllable labels to the yaml file.

```{note}


In [ ]:
# create index dataframe and save it to csv
syll_info_df = kpms.create_syll_info_file(project_dir, model_name)
# display first 5 row of syllable info dataframe
syll_info_df.head()

In [ ]:
# update information from csv to index file
kpms.update_syll_info_file(project_dir, model_name)

## Compare between groups

Test for statistically significant differences between groups of recordings. The code below takes a syllable property (e.g. frequency or duration), plots its disribution for each syllable across for each group, and also tests whether the property differs significantly between groups. The results are summarized in a plot that is saved to `{project_dir}/{model_name}/analysis_figures`.

There are two options for setting the order of syllables along the x-axis. When `order='stat'`, syllables are sorted by the mean value of the statistic. When `order='diff'`, syllables are sorted by the magnitude of difference between two groups that are determined by the `ctrl_group` and `exp_group` keywords. Note `ctrl_group` and `exp_group` are not related to significance testing.

In [ ]:
kpms.plot_syll_stats_with_sem(
    stats_df, project_dir, model_name,
    plot_sig=True,    # whether to mark statistical significance with a star
    thresh=0.05,      # significance threshold
    stat='frequency', # statistic to be plotted (e.g. 'duration' or 'velocity_px_s_mean')
    order='stat',     # order syllables by overall frequency ("stat") or degree of difference ("diff")
    ctrl_group='a',   # name of the control group for statistical testing
    exp_group='b',    # name of the experimental group for statistical testing
    figsize=(8, 4),   # figure size    
    groups=stats_df['group'].unique(), # groups to be plotted
);

### Transition matrices
Generate heatmaps showing the transition frequencies between syllables.

In [ ]:
normalize='bigram' # normalization method ("bigram", "rows" or "columns")

trans_mats, usages, groups, syll_include=kpms.generate_transition_matrices(
    project_dir, model_name, normalize=normalize,
    min_frequency=0.005 # minimum syllable frequency to include
)    

kpms.visualize_transition_bigram(
    project_dir, model_name, groups, trans_mats, syll_include, normalize=normalize, 
    show_syllable_names=True # label syllables by index (False) or index and name (True)
)

### Syllable Transition Graph
Render transition rates in graph form, where nodes represent syllables and edges represent transitions between syllables, with edge showing transition rate for each pair of syllables (secifically the max of the two transition rates in each direction). The code below first generates a transition graph for each single group, and then generates a difference-graph for each pair of groups.

In [ ]:
kpms.plot_transition_graph_group(
    project_dir, model_name, 
    groups, trans_mats, usages, syll_include, 
    layout='circular',        # transition graph layout ("circular" or "spring")
    show_syllable_names=False # label syllables by index (False) or index and name (True)
)

In [ ]:
kpms.plot_transition_graph_difference(project_dir, model_name, 
                                      groups, trans_mats, usages, syll_include, 
                                      layout='circular') # transition graph layout ("circular" or "spring")